# Satellite imagery collection and processing
---

Experimenting with filtering, downloading and displaying Sentinel 2 images from the [AWS STAC of Cloud-Optimized GeoTIFFs](https://registry.opendata.aws/sentinel-2-l2a-cogs/)

## Setup

### Imports

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
from coal_emissions_monitoring.data_cleaning import load_clean_campd_facilities_gdf, load_clean_campd_emissions_df
from coal_emissions_monitoring.satellite_imagery import get_aws_cog_links_from_geom

### Parameters

In [ ]:
start_date = "2019-01-01"
end_date = "20213-03-31"
buffer_meters = 3200 / 2
# buffer_degrees = buffer_meters / 111000

## Load CAMPD data

In [ ]:
campd_facilities_gdf = load_clean_campd_facilities_gdf("/Users/adminuser/GitHub/ccai-ss23-ai-monitoring-tutorial/data/facility-attributes-2d71649a-2e7f-4fdf-abaa-e0529ce2fc62.csv")
campd_facilities_gdf

In [ ]:
facility_dfs = []
for facility_id in campd_facilities_gdf.facility_id.unique():
    facility_df = campd_facilities_gdf[campd_facilities_gdf.facility_id == facility_id]
    # TODO identify what is the local CRS for the current facility, based on its latitude and longitude
    # check ChatGPT responses
    # convert to the local CRS, based on the coordinates
    # facility_df = facility_df.to_crs(epsg=___)
    # buffer the geometry into a square that is ~3.2km on each side
    facility_df.geometry = facility_df.geometry.buffer(buffer_meters, cap_style=3)
    # convert back to the global CRS
    facility_df = facility_df.to_crs(epsg=4326)
    facility_dfs.append(facility_df)
facility_gdf = gpd.GeoDataFrame(pd.concat(facility_dfs, ignore_index=True))

In [ ]:
# buffer the point geometries to become a square polygon
campd_facilities_gdf["geometry"] = campd_facilities_gdf.to_crs(epsg=3857).buffer(buffer_meters, cap_style=3).to_crs(epsg=4326)
campd_facilities_gdf

In [ ]:
campd_facilities_gdf.geometry.explore()

## Filter emissions data to days when a cloudless image is available

## Download and display images